## Using Gravity Spy pipeline to reclassifying glitches

In [2]:
%matplotlib inline
# %matplotlib qt

In [3]:
!pip install keras
!pip install tensorflow

In [4]:
!pip install gwtrigfind
!pip install panoptes_client

In [5]:
import sys
from pathlib import Path

# Caminho absoluto para a pasta GravitySpy-develop
gravityspy_root = Path("GravitySpy-develop").resolve()

# Adiciona ao sys.path
if str(gravityspy_root) not in sys.path:
    sys.path.append(str(gravityspy_root))

In [6]:
from gwpy.timeseries import TimeSeries
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import requests
import sys

from astropy.time import Time
from gwosc.datasets import event_gps

#FIXME: your path to your Gravity Spy repo
path_to_repo = "C:/Users/erick/Downloads/GW research/glitches/python/GravitySpy-develop/GravitySpy-develop"
sys.path.insert(0, path_to_repo)

from gravityspy.classify import classify
from gravityspy.utils import utils
from gravityspy.plot import plot_qtransform
warnings.filterwarnings('ignore')

In [7]:
# URL of the CSV file
url = "https://zenodo.org/records/5649212/files/H1_O3a.csv?download=1"
file_to_store = './websites data/H1_O3a_gspyLIGO.csv' # Fix this to your desired path and name
# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Write the content of the response to a local CSV file
    with open(file_to_store, "wb") as file:
        file.write(response.content)
    
    glitches = pd.read_csv(file_to_store) # if succesful we load it
    print("File downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

FileNotFoundError: [Errno 2] No such file or directory: './websites data/H1_O3a_gspyLIGO.csv'

In [ ]:
glitches.head()

In [ ]:
j = 17 # second glitch

# 2 Low_Frequency_Burst     17 Koi_Fish
# 19 Extremely_Loud         23 Tomte
# 25 No_Glitch              29 Blip
# 35 Low_Frequency_Lines

event_time = glitches.iloc[j]['peak_time'] + 10**(-9)*glitches.iloc[j]['peak_time_ns']
start = event_time - 20 # we load 20s before and after to avoid border effects
end = event_time + 20

In [ ]:
ifo = glitches.iloc[0]['ifo'] # need to fetch data 
channel_name = f'{ifo}:GDS-CALIB_STRAIN' # to use later
# Now we call the data with gpwy
glitch_series = TimeSeries.fetch_open_data(ifo, start, end)

In [ ]:
config = utils.GravitySpyConfigFile()
plot_time_ranges = config.plot_time_ranges
plot_normalized_energy_range = config.plot_normalized_energy_range

In [ ]:
glitches.iloc[j]

In [ ]:
specsgrams, q_value = utils.make_q_scans(event_time=event_time,
                                         timeseries=glitch_series,
                                         config=config,
                                         channel_name=channel_name)

ind_fig_all, super_fig = plot_qtransform(specsgrams,
                                         plot_normalized_energy_range,
                                         plot_time_ranges,
                                         ifo,
                                         event_time)
super_fig # this combines the 4 different time resolutions of Gravity Spy